In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
def tpr(TP, FN):
    if TP + FN == 0:
        return 1
    return TP / (TP + FN)


def fpr(FP, TN):
    if FP + TN == 0:
        return 0
    return FP / (FP + TN)

In [3]:
from typing import List


def getAUC(x: List[float], y:List[float]) -> float:
    """
    Calculates the area under the curve (AUC) using the trapezoidal rule.
    """
    if len(x) != len(y):
        raise ValueError("The number of x and y values must be equal")

    n = len(x)
    area = 0.0

    for i in range(1, n):
        # Trapezoidal rule: area of trapezoid = (1/2) * (sum of parallel sides) * height
        area += 0.5 * (x[i] - x[i-1]) * (y[i] + y[i-1])

    return area // 0.001 / 1000

In [4]:
def getFPRAndTPR(dataset, model_name, criteria):
    eval_res_df = (
        pd.read_csv(
            f"./results/{dataset}/{model_name}/criteria{criteria}.csv",
            header=0,
        )
        .sort_values("threshold")
        .reset_index(drop=True)
    )

    if criteria == 2:
        eval_res_df["FN"] = eval_res_df["FN"] + eval_res_df["TP_LATE"]

    eval_res_df["TPR"] = eval_res_df.apply(
        lambda row: tpr(row["TP"], row["FN"]), axis=1
    )
    eval_res_df["FPR"] = eval_res_df.apply(
        lambda row: fpr(row["FP"], row["TN"]), axis=1
    )
    
    eval_res_df = eval_res_df.sort_values("FPR").reset_index(drop=True)

    fin_tpr = eval_res_df["TPR"].tolist()
    fin_tpr.insert(0, 0)
    # fin_tpr.append(1)
    
    fin_fpr = eval_res_df["FPR"].tolist()
    fin_fpr.insert(0, 0)
    # fin_fpr.append(1)

    return fin_fpr, fin_tpr

In [5]:
def generateROCForCriteria(dataset: str, criteria: int, models = ["numenta", "knncad", "expose", "ARTime"]):

    fig = go.Figure()

    for model_name in models:
        fin_fpr, fin_tpr = getFPRAndTPR(dataset, model_name, criteria)

        fig.add_trace(go.Scatter(x=fin_fpr, y=fin_tpr, name=f"{model_name } - AUC: {getAUC(fin_fpr, fin_tpr)}"))

    fig.add_trace(
        go.Scatter(x=[0, 1], y=[0, 1], line_color="lightgray", name="Baseline")
    )

    fig.update_layout(
        title=f"ROC Curve - {dataset} - criteria: {criteria}",
        xaxis_title="False Positive Rate",
        yaxis_title="True Positive Rate",
        yaxis_range=[0, 1.1],
        xaxis_range=[0, 1],
        height=400,
        width=550,
    )
    fig.update_yaxes(
        scaleanchor="x",
        scaleratio=1,
    )

    style_dict = {
        'layout.plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'layout.font.family': 'Arial',
        'layout.font.size': 10,
        'layout.xaxis.linecolor': 'black',
        'layout.xaxis.ticks': 'inside',
        'layout.xaxis.mirror': True,
        'layout.xaxis.showline': True,
        'layout.yaxis.linecolor': 'black',
        'layout.yaxis.ticks': 'inside',
        'layout.yaxis.mirror': True,
        'layout.yaxis.showline': True,
        # 'layout.autosize': False,
        'layout.showlegend': True,
        'layout.legend.bgcolor': 'rgba(0, 0, 0, 0)',
        'layout.legend.xanchor': 'right',
        # 'layout.legend.x': 1,
        # 'layout.legend.font.family': 'monospace',
        # Specialized:
        # 'layout.xaxis.range': (2.3, 2.5),
        # 'layout.yaxis.range': (-50, +50),
        # 'layout.xaxis.title': r'$x$',
        # 'layout.yaxis.title': r'$y$',
        # 'layout.title': 'Advanced Example of a Line Plot with Plotly',
    }
    fig.update(**style_dict)

    fig.show()

In [6]:
config = {
        "vmft-lad_Fake": { "name": "VMFT-LAD no feedback", "color": "#a6cee3"},
        "vmft-lad_GPT3": { "name": "VMFT-LAD w/ GPT 3.5 turbo", "color": "#1f78b4"},
        "vmft-lad_Falcon7B": { "name": "VMFT-LAD w/ Falcon 7B", "color": "#6a3d9a"},
        "vmft-lad_Cyrax7B": { "name": "VMFT-LAD w/ Cyrax 7B", "color": "#2b758e"},
        "vmft-lad_EmertonMonarch7B": { "name": "VMFT-LAD w/ Emerton Monarch 7B", "color": "#cab2d6"},
        "vmft-lad_BartLarge": { "name": "VMFT-LAD w/ Bart Large (Zero Shot)", "color": "#ff7f00"},
        "numenta": { "name": "HTM", "color": "#e31a1c"},
        "knncad": { "name": "KNN-CAD", "color": "#b15928"},
        "expose": { "name": "EXPOSE", "color": "#33a02c"},
        "ARTime": { "name": "ARTime", "color": "#fb9a99"}
    }

In [7]:
def generateROCForDataset(models = ["vmft-lad_Fake", "vmft-lad_GPT3", "numenta", "knncad", "expose", "ARTime"], filename="ROC_all_comp_models.png"):

    # config = {
    #     "vmft-lad": { "name": "VMFT-LAD", "color": "#636efa"},
    #     "numenta": { "name": "HTM", "color": "#ab63fa"},
    #     "knncad": { "name": "KNN-CAD", "color": "#ff6692"},
    #     "expose": { "name": "EXPOSE", "color": "#00cc96"},
    #     "ARTime": { "name": "ARTime", "color": "#ff97ff"}
    # }

    dataset_config = {
        "hdd": { "name": "HDD", "row": 1, "col": 1},
        "cpu": { "name": "CPU", "row": 1, "col": 3},
        "oom": { "name": "OOM", "row": 2, "col": 1},
        "buffer-io": { "name": "Buffer I/O", "row": 2, "col": 3}
    }
    
    

    fig = make_subplots(
        rows=2, cols=4,
        column_widths=[0.25, 0.25, 0.25, 0.25],
        # row_heights=[0.5],
        subplot_titles=("HDD - Criteria 1", "HDD - Criteria 2",
                        "CPU - Criteria 1", "CPU - Criteria 2",
                        "OOM - Criteria 1", "OOM - Criteria 2",
                        "Buffer I/O - Criteria 1", "Buffer I/O - Criteria 2"),
        horizontal_spacing = 0.08,
        vertical_spacing = 0.18,
    )
    
    for dataset in dataset_config.keys():
        for criteria in [1, 2]:
            fig.add_trace(
                go.Scatter(x=[0, 1], y=[0, 1], line_color="lightgray", name="Baseline", 
                        #    legendgroup='group1', 
                           showlegend=False),
                row=dataset_config[dataset]["row"], col=(dataset_config[dataset]["col"] + (criteria - 1))
            )
            for model_name in models:
                fin_fpr, fin_tpr = getFPRAndTPR(dataset, model_name, criteria)

                fig.add_trace(
                    go.Scatter(
                                x=fin_fpr, y=fin_tpr, 
                                name=config[model_name]['name'], 
                                line_color=config[model_name]['color'], 
                                # legendgroup='group1', 
                                showlegend=dataset=='buffer-io' and criteria==2,
                                # mode='lines+markers',
                                # marker_size=5
                            ), 
                            row=dataset_config[dataset]["row"], col=(dataset_config[dataset]["col"] + (criteria - 1))
                )
        

    fig.update_layout(
        template='simple_white',
        height=760,
        width=1448,
        font=dict(
            family="Arial",
            size=18,
            color="black"
        ),
        # title=f"ROC Curve - {dataset}",
        # title_yanchor="bottom",
        # title_xanchor="center",
        title_xref="paper",
        title_y=0.05,
        title_x=0.5,
        # title_pad=0
        # pad=5,
        # legend=dict(
        #     x=0.98,
        #     y=0.05,
        #     xanchor='center',
        #     yanchor='bottom',
        #     bgcolor= 'rgba(0, 0, 0, 0)'
        # ),
        legend={
            "orientation":"h",
        },
        margin=dict(
            t=40,
            r=50,
            l=60,
            b=40,
            # autoexpand=False
        )
    )
    # fig.update_yaxes(
    #     scaleanchor="x",
    #     scaleratio=1,
    # )

    fig.update_annotations(font_size=19)
    
    for row in range(1, 3):
        for col in range(1, 5):
            fig.update_xaxes(title_text="False Positive Rate", range=[-0.01, 1.01], row=row, col=col, title_standoff=5)
            fig.update_yaxes(title_text="True Positive Rate", range=[-0.01, 1.01], row=row, col=col, title_standoff=5)

    fig.write_image(f"images/{filename}",scale=4, width=1448, height=760)

    fig.show()

In [8]:
generateROCForDataset(
["vmft-lad_Fake",
"vmft-lad_GPT3",
"vmft-lad_Falcon7B",
"vmft-lad_Cyrax7B",
"vmft-lad_EmertonMonarch7B",
"vmft-lad_BartLarge",
"ARTime",],  filename="ROC_all_vmft-lad.png")

In [9]:
generateROCForDataset()

In [10]:
def getAUCForDataset(dataset: str, models = config.keys()):
    # for model_name in models:
    #     print(f"{model_name}\t", end="")
    # print()
    for criteria in [1, 2]:
        # print(f"Criteria {criteria}")
        for model_name in models:
            fin_fpr, fin_tpr = getFPRAndTPR(dataset, model_name, criteria)
            print(f"{getAUC(fin_fpr, fin_tpr)}")
        # print()


In [11]:
getAUCForDataset("hdd")

0.992
0.999
0.996
0.992
0.996
0.991
0.988
0.684
0.512
0.997
0.987
0.999
0.993
0.989
0.995
0.987
0.987
0.659
0.512
0.973


In [12]:
getAUCForDataset("buffer-io")

0.993
0.999
0.978
0.997
0.993
0.993
0.99
0.607
0.501
0.995
0.992
0.999
0.962
0.997
0.992
0.993
0.989
0.607
0.501
0.995


In [13]:
getAUCForDataset("oom")

0.903
0.999
0.965
0.974
0.909
0.836
0.977
0.626
0.499
0.986
0.657
0.998
0.767
0.899
0.667
0.621
0.623
0.304
0.499
0.546


In [14]:
getAUCForDataset("cpu")

0.678
0.999
0.678
0.754
0.817
0.725
0.888
0.677
0.511
0.78
0.588
0.996
0.588
0.594
0.669
0.634
0.68
0.664
0.511
0.458
